# 인공지능 비트 트레이더 경진대회 시즌3

### 2위 ARIMA

- ARIMA 에 VWAP 및 피쳐 엔지니어링을 통해 생성한 값 diff (open - VWAP)를 학습 데이터로 사용
- 안전에 집중
    - 수익을 많이 거두어도 나중에 잘못된 타이밍에 매도를 하게 되면, 복리에 하락이 겹쳐 큰 손해 발생


In [ ]:
# ARIMA 모델을 사용하기 위해 statsmodels 제일 최신 버전을 설치
!pip install statsmodels==0.12.2

In [ ]:
# 1. 기본
# pandas와 numpy를 import 
import pandas as pd
import numpy as np


# 2. 시각화
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


# 3. 유틸
# tqdm 패키지는 반복문에 대해 얼마나 진척되었는지를 가시적으로 확인
# https://github.com/tqdm/tqdm 사용법 참고
from tqdm.auto import tqdm


# 4. 경고 설정
import warnings
warnings.filterwarnings('ignore')


# 5. stats models
# 시계열 모델을 위한 ARIMA
from statsmodels.tsa.arima_model import ARIMA


# 6. 구글 드라이브
from google.colab import drive



In [ ]:
### 데이터를 다운 받지 못함
### 대회 마감으로 참가 버튼이 활성화되지 않아 데이터를 다운 받지 못하였습니다.

In [ ]:
# train_x, train_y 를 sample_id 기준 하나로 합쳐서, train_z를 만듦
# train x, y를 합치지 않아도 괜찮으나, 연속되는 변수 vwap, rsi, 등등을 만들기 위해 x, y를 합치고, 변수를 생성하고, x, y를 다시 분리하는 작업을 진행
train_x["is_x"] = 1
train_y["is_x"] = 0
train_x_y = [train_x, train_y]
train_list = [x.set_index('sample_id') for x in train_x_y]

train_z = pd.concat(train_list, axis=0).rename_axis('sample_id').reset_index()

**VWAP**

거래량 가중 평균가(VWAP, Volume Weighted Average Price)는 거래량에 가중치가 부여되어 특정 기간 동안의 평균 가격을 의미
- 트레이딩에서 VWAP은 상승세와 하락세를 판단하는 지표로 사용
- 가격(open)이 vwap보다 크면 상승세로 판단하고, 만약 가격이 VWAP 보다 작으면 하락세로 판단

VWAP = sum(Volume*Price)/sum(Volume)

* 참고 링크: https://academy.binance.com/ko/articles/volume-weighted-average-price-vwap-explained

**Key Pint**
- 가격에서 VWAP을 뺀 값인 diff (open - VWAP)을 학습에 사용, 2가지 이점
    * 안전한 지점을 찾을 수 있는 것 
    * open - vwap이 양수이면, 상승 구간에 있기 때문에 안전한 지점을 찾을 수 있고, 적어도 손해는 안 볼 확률이 높아진다고 판단
    * 제일 가격이 높은 지점을 찾는 것이 아닌 안전한 지점을 찾는 문제로 바꿈


- 2개의 변수를 한 번에 사용할 수 있는 것
    * 개인적으로 여러 강의와 코드 샘플들을 보았지만, 변수가 저렇게 많은데 단 1가지 feature만 학습에 사용
    * 적어도 2개 이상의 변수를 함께 사용하면 조금 더 좋은 예측을 할것이라 생각

In [ ]:
# vwap과, open에서 vwap을 뺀 값인 diff를 계산합니다.
def make_vwap_and_diff(df):

    # 1) VAWP 계산

    # 일반적인 VWAP 공식에서 volume을 그대로 사용하지만, 여러번의 시도를 통해 tb_base_av 와 volume을 더했을 때 가장 좋은 volume이 나온다고 판단하영 사용하였습니다.
    df["volume_tb_base_av"] = df["tb_base_av"] + df["volume"]

    # open하나만을 사용하기 보다는 open(시가), high(고가), low(저가) 3개의 평균을 price로 사용하였습니다.
    df['volume_price'] = ((df['open'] + df['high'] + df['low']) / 3) * df['volume_tb_base_av']

    # price와 volume의 곱의 합을 구해줍니다.
    df['volume_price_sum'] = df.groupby(['sample_id'])['volume_price'].apply(lambda x: x.cumsum())

    # volume의 합을 구해줍니다.
    df['volume_sum'] = df.groupby(['sample_id'])['volume_tb_base_av'].apply(lambda x: x.cumsum())

    # 2 변수의 나눗셈을 통해 vwap을 계산해줍니다.
    df['vwap'] = df['volume_price_sum'] / df['volume_sum']


    # 2) diff 계산
    # 매도수익이 open을 통해 이루어진다고 알려져있기 때문에 open에서 vwap을 뺀 값을 diff로 사용했습니다.
    df["diff"] = df["open"] - df["vwap"]


    return df
    

In [ ]:
# 데이터 프레임에서 sample_id 에 따른 open을 반환하는 함수입니다.
def get_open(df,sample_id):
    
    return df[df["sample_id"] == sample_id]['open'].values
    

In [ ]:
# 데이터 프레임에서 sample_id 에 따른 VWAP을 반환하는 함수입니다.
def get_vwap(df,sample_id):
    
    return df[df["sample_id"] == sample_id]['vwap'].values
    

In [ ]:
# 데이터 프레임에서 sample_id 에 따른 diff 반환하는 함수입니다.
def get_diff(df,sample_id):
    
    return df[df["sample_id"] == sample_id]['diff'].values
    

In [ ]:
# 데이터 프레임에서 sample_id 에 따른 rsi 반환하는 함수입니다.
def get_rsi(df,sample_id):
    
    return df[df["sample_id"] == sample_id]['rsi'].values
    

In [ ]:
# 데이터 프레임에서 sample_id 에 따른 col_name 값을 반환하는 함수입니다.
def get_series(df,sample_id, col_name):
    
    return df[df["sample_id"] == sample_id][col_name].values
    

**RSI**
- 상대강도지수(relative strength index)는 가격의 상승압력과 하락압력 간의 상대적인 강도를 나타냄
    - 트레이딩에서 사용
    - RSI 값이 30보다 작으면 초과매도로 판단하고, RSI 값이, 70 보다 크면 초과매수 상태로 판단
    - 최고점과, 최하점을 찾기 쉽다는 장점
RSI = AU/(AU+AD)



- 천정과 바닥이 제대로 형성되지 않은 시장(RSI가 50근처 유지)에서는 유용하지 못함
- 코인 데이터의 RSI 그래프를 그려보면 RSI 0 ~ 100 사이를 급변동

* 안정적으로 투자하기 위해 65 초과인 상태를 초과매수국면으로 판단하고, 해당 시점 이후 50분동안은 투자하지 않도록 설정

In [ ]:
# RSI를 만들어줍니다.
# 상승분, 하락분의 평균은 일반적으로 14일을 기준으로 생성합니다.
def make_rsi(df, period=14):

    # 전일 대비 상승분을 계산해줍니다. - 상승분이 0보다 크면 상승분을 넣고, 0보다 작거나 같으면 0을 넣어줍니다.
    df["U"] = np.where(df.groupby(["sample_id"])["open"].diff(1) > 0, df.groupby(["sample_id"])["open"].diff(1), 0)

    # 전일 대비 하락분을 계산해줍니다. - 하락분이 0보다 작으면 하락분 * -1을 넣고, 0보다 크거나 같으면 0을 넣어줍니다.
    df["D"] = np.where(df.groupby(["sample_id"])["open"].diff(1) < 0, df.groupby(["sample_id"])["open"].diff(1) *(-1), 0)


    # 전일 대비 상승분의 평균을 계산해줍니다.
    ud_df = pd.DataFrame()
    ud_df["sample_id"] = df["sample_id"]
    ud_df["U"] = df["U"]
    ud_df["D"] = df["D"]

    # 상승분의 14일 평균을 구해줍니다.
    df["AU"] = ud_df.groupby(["sample_id"])["U"].rolling( window=period, min_periods=period ).mean().reset_index()["U"]
    # 하락분의 14일 평균을 구해줍니다.
    df["AD"] = ud_df.groupby(["sample_id"])["D"].rolling( window=period, min_periods=period ).mean().reset_index()["D"]


    # AU / (AU + AD) 의 백분율을 RSI 로 계산해줍니다.
    RSI = df["AU"] / (df["AU"] + df["AD"]) * 100
    
    df["rsi"] = RSI
    
    return df
    

In [ ]:
### 전처리 ###
# 1. train, test의 sample_id 목록을 저장합니다.
TRAIN_SAMPLE_ID_LIST = train_x["sample_id"].unique().tolist()
TEST_SAMPLE_ID_LIST = test_x["sample_id"].unique().tolist()

# 2. VWAP, diff 를 만들어줍니다.
test_x = make_vwap_and_diff(test_x)
train_z = make_vwap_and_diff(train_z)

# 3. rsi 를 만들어줍니다.
test_x = make_rsi(test_x, 14)
train_z = make_rsi(train_z, 14)

# 4. train x와 y를 분리합니다.
train_x = train_z[train_z["is_x"] == 1]
train_y = train_z[train_z["is_x"] == 0]

split_drop_cols = ["is_x"]

train_x = train_x.drop(columns=split_drop_cols, axis=1)
train_y = train_y.drop(columns=split_drop_cols, axis=1)



**모델학습**

1) 모델
- 모델은 diff (open - vwap)을 ARIMA를 통해 학습하여 생성


2) 제약조건
- vwap, rsi의 마지막 값을 제약조건의 기준으로 사용
- vwap이 1보다 크면 open보다 vwap이 크다는 의미로 하향세에 접어들었다고 판단 투자하지 않음 (x의 open 의 마지막 값은 1입니다.)


*rsi의 값이 65보다 크면 초과 매수 상태라고 판단하여 투자하지 않았습니다. 기본적으로 70 초과로 판단하는데 임의로 바꿀수 있는 값으로 조금 낮춰서 65로 시도했을때가 제일 좋아서 사용했습니다.*


3) ARIMA와 p d q
- 개인적으로 ARIMA 모델의 order(p, d, q)에 투자를 많이 함
- ARIMA 모델의 AIC (아카이케 정보 기준) 가 가장 나오는 order를 brute force로 찾아보았지만, 2시간 30분 ~ 3시간이 걸림에도 불구하고, 점수가 오히려 낮아져서, 여러번의 시도로 4, 0, 1 이 제일 좋다고 판단

pac, acf 분석을 통해 AR 2 모델이라고 판단했습니다. 분석에는 다음 영상을 참고했습니다. (https://www.youtube.com/watch?v=-vSzKfqcTDg&t=360s)
*이탤릭체 텍스트*


In [ ]:
result = []

for sample_id in tqdm(TEST_SAMPLE_ID_LIST):

    # 1. 데이터 로드
    # 1) diff - 학습에 사용
    diff_x = get_diff(test_x, sample_id)

    # 2) vwap - 보조 지표로 사용
    vwap_series = get_vwap(test_x, sample_id)

    # 3) rsi - 보조 지표로 사용
    rsi_series = get_rsi(test_x, sample_id)



    # 2. ARIMA
    # 1) 모델 정의
    ARIMA_MODEL = {}
    ARIMA_MODEL_FIT = {}

    # 2) AR 모델 적용
    try:
      ARIMA_MODEL = ARIMA(diff_x, order = (4,0,1))
      ARIMA_MODEL_FIT = ARIMA_MODEL.fit(trend = 'nc', full_output = True, disp = True)

    # 3) 수렴하지 않을 경우 p d q 를 1, 1, 0으로 사용
    except:
      ARIMA_MODEL = ARIMA(diff_x, order = (1,1,0))
      ARIMA_MODEL_FIT = ARIMA_MODEL.fit(trend = 'nc', full_output = True, disp = True)

    # 4) ARIMA 예측
    ARIMA_FORECAST  = ARIMA_MODEL_FIT.predict(1,120, typ='levels')



    # 3. 데이처 처리
    # 1) 최대 부분인 인덱스를 찾는데 해당 시점에 매도를 진행합니다.
    sell_time = np.argmax(ARIMA_FORECAST)

    # 2) 최대값을 찾습니다.
    max_val = np.max(ARIMA_FORECAST)
    
    # 3) vwap의 마지막 값을 가져옵니다.
    vwap_last_val = vwap_series[1379]

    rsi_last_val = rsi_series[1379]



    # 4. 투자 전략
    buy_quantity = 0

    # 1) 최대값이 0 보다 크면 가격이 vwap 보다 크다는 의미로, 투자합니다.
    if  max_val > 0:
        buy_quantity = 1


    # 2) 만약 vwap 마지막 값이, 1보다 크면 가격이 1보다 작다는 의미로 하향세이기 때문에 투자하지 않습니다.
    if vwap_last_val > 1 and sell_time < 50:
        buy_quantity = 0

    # 3) 만약 rsi의 값이 65 보다 크면, 초과매수 상태로 판단하여 투자하지 않습니다.
    if rsi_last_val > 65 and sell_time < 50:
        buy_quantity = 0



    # 5. 결과
    result_list = [
                   sample_id,
                   buy_quantity,
                   sell_time
                  ]

    result.append(result_list)
    

In [ ]:
### 제출 및 확인 ###
# 1. 학습 결과를 데이터 프레임으로 만듭니다.

submit_columns = [
                  "sample_id", 
                  "buy_quantity", 
                  "sell_time"
                  ]


submit = pd.DataFrame(data=result, columns=submit_columns)

# 2. 결과 데이터 프레임 확인

submit.head(10)

# 3. 투자 개수 확인

submit[submit["buy_quantity"] == 1].shape[0]

# 4. sell_time 50미만에서 구매하는 개수 확인
cond1 = (submit["buy_quantity"] == 1)
cond2 = (submit["sell_time"] < 50)

submit[cond1 & cond2].shape[0]

# 5. 제출
# 파일의 이름을 지정해줍니다.
FILE_NAME = "/0603_ARIMA_DIFF_VWAP_RSI_65_UNDER_50_SUBMIT.csv"

# 제출경로에 파일을 생성해줍니다.
RESULT_PATH = SUBMIT_PATH + FILE_NAME

submit.to_csv(RESULT_PATH, index=False)



**public score를 다음과 같이 활용**

- open에 vwap을 섞어 투자개수가 줄었음에도 점수가 올라가는 현상
- vwap, rsi등 보조지표를 통해 sell_time 10, 20.. 50 미만은 투자하지 않기로 결정했음에도 점점 점수가 올라가는 현상
- rsi 초과매수로 상태를 70 초과가 아닌, 65 초과로, 더 안정적으로 결정해도 점수가 상승하는 현상

이렇게, 제약사항을 통해 보수적으로 투자했음에도 불구하고, 점수가 올라가는 경우에 집중하여, 해당 피쳐가 안정적으로 작용함을 판단

여러 아이디어

- RANDOM BOX 분류 모델
- open에 vwap을 섞은것에 더하여, RSI, 이동평균선등을 모두 섞어 만든 혼합 데이터
- prophet, neural prophet, LSTM등 다양한 모델을 사용
- 위 모델들의 결과를 bagging하는 방법을 사용해보았지만, ARIMA 단 하나만을 사용했을때보다 예측 성능이 낮아짐

### 3위 Fbprophet

- fbprophet에 grid search를 사용해 hyperparameter optimization함
- 하나의 seasonality를 사용하는 것보다 여러 개를 사용하는 것이 validation 과정에서 MAPE에 좋은 영향을 주었기 때문에 seasonality를 추가하며 파라미터를 조정
- 돈을 잃지 않는 것을 목적으로 threshold를 높게 잡았고(샘플 중 약 3%를 매수), 100%의 매수 방식을 유지
- 전처리는 이미 scaling이 되어 있기 때문에 따로 진행을 하지 않음
- rolling MA를 추가적으로 사용해서 보다 안정적인 예측을 해보는 것도 좋을 것 같음

!pip install -q fbprophet==0.7.1

In [ ]:
### Library Import ###
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import random
from dateutil import parser
from tqdm import tqdm
from fbprophet import Prophet
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
start_time = '2021-01-31 00:00:00'
start_dt = datetime.datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')

In [ ]:
data_path = '.'
train_x_df = pd.read_csv(data_path  + "/train_x_df.csv")
train_y_df = pd.read_csv(data_path  + "/train_y_df.csv")
test_x_df = pd.read_csv(data_path  + "/test_x_df.csv")

In [ ]:
def df2d_to_array3d(df_2d):
    # 입력 받은 2차원 데이터 프레임을 3차원 numpy array로 변경하는 함수
    feature_size = df_2d.iloc[:,2:].shape[1]
    time_size = len(df_2d.time.value_counts())
    sample_size = len(df_2d.sample_id.value_counts())
    sample_index = df_2d.sample_id.value_counts().index
    array_3d = df_2d.iloc[:,2:].values.reshape([sample_size, time_size, feature_size])
    return array_3d

In [ ]:
train_x_array = df2d_to_array3d(train_x_df)
train_y_array = df2d_to_array3d(train_y_df)
test_x_array = df2d_to_array3d(test_x_df)

In [ ]:
print(f'''
train_x_array {train_x_array.shape}
train_y_array {train_y_array.shape}
test_x_array {test_x_array.shape}
''')

In [ ]:
### EDA ###
def plot_series(x_series, y_series):
    #입력 series와 출력 series를 연속적으로 연결하여 시각적으로 보여주는 코드 입니다.
    plt.plot(x_series, label = 'input_series')
    plt.plot(np.arange(len(x_series), len(x_series)+len(y_series)),
             y_series, label = 'output_series')
    plt.axhline(1, c = 'red')
    plt.legend()

idx = 42
plot_series(train_x_array[idx,:,1], train_y_array[idx,:,1])
plt.show()

In [ ]:
### Parameter Optimization ###
# fbprophet 모델에 MAPE를 기준으로 grid searching을 적용
params_grid = {'changepoint_prior_scale':[0.1,0.01,0.001],
               'n_changepoints' : [50,100,150],
               'fourier_order' : [5,10],
               'period':[0.1,0.3,0.5]
               }
grid = ParameterGrid(params_grid)
cnt = 0
for p in grid:
    cnt = cnt+1

print('Total Possible Models',cnt)

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
# MAPE가 최소가 되는 값을 찾고, 추가적으로 2개의 seasonality를 더해서 모델 성능 높임

model_parameters = pd.DataFrame(columns = ['idx',
                                           'MAPE',
                                           'changepoint_prior_scale',
                                           'n_changepoints',
                                           'fourier_order',
                                           'period'
                                           ])

random.seed(42)
samples = random.sample(range(7661),10)
for idx in tqdm(samples, position=1):
    x_series = train_x_array[idx,:,1]
    y_series = train_y_array[idx,:,1]
    x_df = pd.DataFrame()
    x_df['ds'] = [start_dt + datetime.timedelta(minutes = time_min) for time_min in np.arange(1, x_series.shape[0]+1).tolist()]
    x_df['y'] = x_series.tolist()
    for p in tqdm(grid, position=0):
        prophet = Prophet(seasonality_mode='multiplicative', 
                          yearly_seasonality=False,
                          weekly_seasonality=False,
                          daily_seasonality=False,
                          changepoint_prior_scale=p['changepoint_prior_scale'],
                          n_changepoints=p['n_changepoints'],
                          )
        prophet.add_seasonality(name='seasonality_1',period=p['period'],fourier_order=p['fourier_order'])
        prophet.fit(x_df)

        # 120분 테스트 데이터를 예측합니다.
        future_data = prophet.make_future_dataframe(periods=120, freq='min')
        forecast_data = prophet.predict(future_data)

        pred_y = forecast_data.yhat.values[-120:]

        MAPE = mean_absolute_percentage_error(y_series,pred_y)
        model_parameters = model_parameters.append({'idx':idx,
                                                    'MAPE':MAPE,
                                                    'changepoint_prior_scale':p['changepoint_prior_scale'],
                                                    'n_changepoints':p['n_changepoints'],
                                                    'fourier_order':p['fourier_order'],
                                                    'period':p['period']
                                                    },ignore_index=True)

In [ ]:
idx = 42
x_series = train_x_array[idx,:,1]
  
x_df = pd.DataFrame()
x_df['ds'] = [start_dt + datetime.timedelta(minutes = time_min) for time_min in np.arange(1, x_series.shape[0]+1).tolist()]
x_df['y'] = x_series.tolist()

prophet = Prophet(seasonality_mode='multiplicative', 
          yearly_seasonality=False,
          weekly_seasonality=False,
          daily_seasonality=False,
          changepoint_prior_scale=0.1,
          n_changepoints=50
          )
prophet.add_seasonality(name='seasonality_1',period=0.1,fourier_order = 5)
prophet.add_seasonality(name='seasonality_2', period=0.3,fourier_order = 5)
prophet.add_seasonality(name='seasonality_3', period=0.01, fourier_order = 5)
prophet.fit(x_df)

future_data = prophet.make_future_dataframe(periods=120, freq='min')
forecast_data = prophet.predict(future_data)

pred_y = forecast_data.yhat.values[-120:]
pred_y_lower = forecast_data.yhat_lower.values[-120:]
pred_y_upper = forecast_data.yhat_upper.values[-120:]

In [ ]:
plt.plot(np.hstack([train_x_array[idx,:,1], train_y_array[idx,:,1]]), label='true')
plt.plot(np.hstack([train_x_array[idx,:,1], pred_y]), label='prediction')
plt.legend()
plt.axvline(1380, c='red')

In [ ]:
### Training ###
# 베이스라인 코드에 있었던 100%의 매수 방식을 그대로 유지. threshold는 상위 3% 정도를 매수하도록 조정
test_pred_array = np.zeros([test_x_array.shape[0],
                           120])
for idx in tqdm(range(test_x_array.shape[0]), position=0):
    
    x_series = test_x_array[idx,:,1]
  
    x_df = pd.DataFrame()
    x_df['ds'] = [start_dt + datetime.timedelta(minutes = time_min) for time_min in np.arange(1, x_series.shape[0]+1).tolist()]
    x_df['y'] = x_series.tolist()

    prophet = Prophet(seasonality_mode='multiplicative', 
              yearly_seasonality=False,
              weekly_seasonality=False,
              daily_seasonality=False,
              changepoint_prior_scale=0.1,
              n_changepoints=50
              )
    prophet.add_seasonality(name='seasonality_1',period=0.1,fourier_order = 5)
    prophet.add_seasonality(name='seasonality_2', period=0.3,fourier_order = 5)
    prophet.add_seasonality(name='seasonality_3', period=0.01, fourier_order = 5)
    prophet.fit(x_df)

    # 120분 테스트 데이터를 예측합니다.
    future_data = prophet.make_future_dataframe(periods=120, freq='min')
    forecast_data = prophet.predict(future_data)

    pred_y = forecast_data.yhat.values[-120:]

    test_pred_array[idx,:] = pred_y

In [ ]:
def array_to_submission(x_array, pred_array):
    # 입력 x_arrry와 출력 pred_arry를 통해서 
    # buy_quantitiy와 sell_time을 결정
    submission = pd.DataFrame(np.zeros([pred_array.shape[0],2], np.int64),
                columns = ['buy_quantity', 'sell_time'])
    submission = submission.reset_index()
    submission.loc[:, 'buy_quantity'] = 0.1
    
    buy_price = []
    for idx, sell_time in enumerate(np.argmax(pred_array, axis = 1)):
        buy_price.append(pred_array[idx, sell_time])
    buy_price = np.array(buy_price)
    
    submission.loc[:, 'buy_quantity'] = (buy_price > 1.07) * 1
    submission['sell_time'] = np.argmax(pred_array, axis = 1)
    submission.columns = ['sample_id','buy_quantity', 'sell_time']
    submission['sample_id'] = submission['sample_id'].apply(lambda x: x+7661)
    return submission

In [ ]:
### Submit ###
submission = array_to_submission(test_x_array, test_pred_array)
submission['buy_quantity'].value_counts()

submission.to_csv('/content/gdrive/My Drive/data/bit_trader_season2.csv', index=False)